### Integration 

We want to merge all of our 26 samples into one AnnData object.

In [5]:
import scvi
import numpy as np
import scanpy as sc
import pandas as pd
from pathlib import Path

ribo_url = "http://software.broadinstitute.org/gsea/msigdb/download_geneset.jsp?geneSetName=KEGG_RIBOSOME&fileType=txt"
ribo_genes = pd.read_table(ribo_url, skiprows=2, header=None)


def preprocess(csv_path: Path):
    # Since scapy expects data to be in cellXgene format,
    # we'll have to transpose our geneXcell csv obtained from GSE171524
    adata = sc.read_csv(csv_path).T
    adata_original = adata.copy()
    # We want to filter out genes that do not appear in more than 10 cells
    # Genes expressed in extremely few cells are more likely to be noise than
    # anything of biological siginificane; could be ambient RNA or seq artifacts
    sc.pp.filter_genes(adata, min_cells=10)
    # Next, we only want to keep the genes with significant
    # biological variance and remove genes that are uninformative
    sc.pp.highly_variable_genes(
        adata, n_top_genes=2000, subset=True, flavor="seurat_v3"
    )
    # We only want to remove doublets
    # We take a probabilistic approach for this
    # And all previous filtering was done to prepare for this step
    # since this is computationally expensive and we'd like to eliminate unncessary compute
    # Read: https://www.sciencedirect.com/science/article/pii/S2405471220301952
    scvi.model.SCVI.setup_anndata(adata)
    vae = scvi.model.SCVI(adata)
    vae.train()
    solo = scvi.external.SOLO.from_scvi_model(vae)
    solo.train()
    df = solo.predict()
    df["prediction"] = solo.predict(soft=False)
    doublets = df[df.prediction == "doublet"]

    adata_original.obs["Sample"] = csv_path.split("_")[
        2
    ]  #'data/GSM5226574_C51ctr_raw_counts.csv'
    adata_original.obs["doublet"] = adata_original.obs.index.isin(doublets.index)
    # We finally remove the doublets from our original matrix
    adata_original = adata_original[~adata_original.obs.doublet]

    # As a next step of filtering, we'd like to get
    # rid of cells that express fewer than 200 genes
    sc.pp.filter_cells(adata_original, min_genes=200)
    # also get rid of genes that appear
    # in fewer than 3 cells; this cuts out possible noise
    sc.pp.filter_genes(adata_original, min_cells=3)
    # Next, we mark mito genes
    adata_original.var["mt"] = adata_original.var_names.str.startswith("mt-")
    # and the ribo genes
    adata_original.var["ribo"] = adata_original.var_names.isin(ribo_genes[0].values)
    # we calculate our quality control matrix with helpful metrics
    sc.pp.calculate_qc_metrics(
        adata_original,
        qc_vars=["mt", "ribo"],
        percent_top=None,
        log1p=False,
        inplace=True,
    )
    # We only want to look into the 98%ile
    # of cells by gene count so only the cells where
    # a lot of genes are detected; cuts out empty reads from droplets
    upper_lim = np.quantile(adata.obs.n_genes_by_counts.values, 0.98)
    adata_original = adata_original[adata_original.obs.n_genes_by_counts < upper_lim]
    # Further we filter down to cells that show less than 20% mitochondrial genes
    adata_original = adata_original[adata_original.obs.pct_counts_mt < 20]
    # and less than 2% ribosomal genes
    adata_original = adata_original[adata_original.obs.pct_counts_ribo < 2]
    return adata_original
